# Lighthouse + Flow Deck Quick Start

This notebook demonstrates how to use the Crazyflie 2.1 brushless drone with both the Lighthouse Positioning System and Flow Deck for precise positioning and movement control.

**Features:**
- Continuous XYZ state estimates from Lighthouse positioning
- MotionCommander integration for smooth movement
- Hover functionality using Lighthouse position feedback
- Forward movement with real-time position monitoring

**Requirements:**
- Crazyflie 2.1 brushless drone
- Lighthouse Deck
- Flow Deck
- Lighthouse base stations (pre-configured)

Reference Documentation: 
* [Bitcraze User Guide - Logging](www.bitcraze.io/documentation/repository/crazyflie-clients-python/master/userguides/userguide_client/#logging)
* [Lighthouse Position System Home Page (bitcraze)](https://www.bitcraze.io/documentation/system/positioning/ligthouse-positioning-system/)
* [Extended Kalman Filters (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/functional-areas/sensor-to-control/state_estimators/#extended-kalman-filter)
* [State Estimation Logging (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/api/logs/#stateestimate)
* [Logging (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-clients-python/master/userguides/userguide_client/#logging)
* [Lighthouse Positioning System: Dataset, Accuracy, and Precision for UAV Research](https://whoenig.github.io/publications/2021_ICRA-Swarm-workshop_Taffanel.pdf)


## Imports


In [1]:
# Standard libraries
import logging
import sys
import time
import threading
from threading import Event
from dotenv import load_dotenv
import os
import numpy as np

# Crazyflie
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper
from cflib.crazyflie.mem import LighthouseMemHelper

## Configuration and Global Variables


In [ ]:
# Load environment variables
load_dotenv()

# URI to the Crazyflie to connect to
URI = uri_helper.uri_from_env(default=os.getenv("RADIO_URI", 'radio://0/80/2M/E7E7E7E7E8'))

# Movement parameters
DEFAULT_HEIGHT = 1.0  # Default flight height in meters
HOVER_TOLERANCE = 0.05  # Position tolerance for hovering in meters
MOVEMENT_SPEED = 0.3  # Movement speed in m/s
UPDATE_RATE = 50  # Position update rate in Hz

# --- Position tracking global vars ---
current_position = {'x': 0.0, 'y': 0.0, 'z': 0.0}
target_position = {'x': 0.0, 'y': 0.0, 'z': DEFAULT_HEIGHT}
# Create a mutex (mutual-exclusion lock) to ensure thread-safe access to current_position
# (i.e. only one thread reads current_position at a time)
position_lock = threading.Lock()


# Events for deck detection
lighthouse_deck_event = Event()
flow_deck_event = Event()


# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


## Deck Detection Callbacks

In [5]:
def param_deck_lighthouse(_, value_str):
    """
    Callback to check if the Lighthouse deck is attached.
    """
    value = int(value_str)
    if value:
        lighthouse_deck_event.set()
        print('Lighthouse deck is attached!')
    else:
        print('Lighthouse deck is NOT attached!')


def param_deck_flow(_, value_str):
    """
    Callback to check if the Flow deck is attached.
    """
    value = int(value_str)
    if value:
        flow_deck_event.set()
        print('Flow deck is attached!')
    else:
        print('Flow deck is NOT attached!')


## Position Logging and Control

In [6]:
def log_position_callback(timestamp, data, logconf):
    """
    Callback function to receive and store position data from Lighthouse.
    """
    with position_lock:
        current_position['x'] = data['stateEstimate.x']
        current_position['y'] = data['stateEstimate.y']
        current_position['z'] = data['stateEstimate.z']
    
    # Print position for monitoring
    print(f"Position: X={current_position['x']:.3f}, Y={current_position['y']:.3f}, Z={current_position['z']:.3f}")

def get_current_position():
    """
    Get the current position in a thread-safe manner.
    """
    with position_lock:
        return current_position.copy()

def set_target_position(x, y, z):
    """
    Set a new target position.
    """
    global target_position
    target_position = {'x': x, 'y': y, 'z': z}
    print(f"Target set to: X={x:.3f}, Y={y:.3f}, Z={z:.3f}")


def is_at_target_position():
    """
    Check if the drone is within tolerance of the target position.
    """
    pos = get_current_position()
    dx = abs(pos['x'] - target_position['x'])
    dy = abs(pos['y'] - target_position['y'])
    dz = abs(pos['z'] - target_position['z'])
    
    return dx < HOVER_TOLERANCE and dy < HOVER_TOLERANCE and dz < HOVER_TOLERANCE


## Movement Functions


In [ ]:
def hover_at_position(scf, duration=5.0):
    """
    Hover at the current target position for a specified duration.
    """
    print(f"Hovering for {duration} seconds...")
    
    start_time = time.time()
    while time.time() - start_time < duration:
        if is_at_target_position():
            print("Position maintained within tolerance")
        else:
            pos = get_current_position()
            print(f"Position drift: X={pos['x']-target_position['x']:.3f}, Y={pos['y']-target_position['y']:.3f}, Z={pos['z']-target_position['z']:.3f}")
        
        time.sleep(0.1)


def move_forward_with_position_monitoring(scf, distance=0.5):
    """
    Move forward while continuously monitoring position.
    """
    print(f"Moving forward {distance}m while monitoring position...")
    
    # Get starting position
    start_pos = get_current_position()
    target_x = start_pos['x'] + distance
    
    # Set new target
    set_target_position(target_x, start_pos['y'], start_pos['z'])
    
    # Move forward using MotionCommander
    with MotionCommander(scf, default_height=start_pos['z']) as mc:
        mc.forward(distance)
        
        # Monitor position during movement
        while not is_at_target_position():
            pos = get_current_position()
            remaining_distance = abs(target_x - pos['x'])
            print(f"Remaining distance: {remaining_distance:.3f}m")
            time.sleep(0.1)
    print("Forward movement completed!")


def takeoff_and_hover(scf, height=DEFAULT_HEIGHT):
    """
    Take off to specified height and hover.
    """
    print(f"Taking off to {height}m and hovering...")
    
    # Get starting position
    start_pos = get_current_position()
    set_target_position(start_pos['x'], start_pos['y'], height)
    
    with MotionCommander(scf, default_height=height) as mc:
        # Take off
        mc.take_off(height)
        
        # Hover for 3 seconds
        hover_at_position(scf, 3.0)
    
    print("Takeoff and hover completed!")


def return_to_start(scf):
    """
    Return to the starting position.
    """
    print("Returning to starting position...")
    
    # Set target to starting position (0, 0, DEFAULT_HEIGHT)
    set_target_position(0.0, 0.0, DEFAULT_HEIGHT)
    
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        # Move to starting position
        pos = get_current_position()
        
        # Calculate required movements
        dx = -pos['x']
        dy = -pos['y']
        
        if abs(dx) > 0.1:
            if dx > 0:
                mc.forward(abs(dx))
            else:
                mc.back(abs(dx))
        
        if abs(dy) > 0.1:
            if dy > 0:
                mc.right(abs(dy))
            else:
                mc.left(abs(dy))
    
    print("Return to start completed!")


## Main Flight Sequence


In [8]:
def main_flight_sequence(scf):
    """
    Main flight sequence demonstrating Lighthouse + Flow Deck integration.
    """
    print("Starting main flight sequence...")
    
    try:
        # 1. Take off and hover
        takeoff_and_hover(scf, DEFAULT_HEIGHT)
        
        # 2. Move forward with position monitoring
        move_forward_with_position_monitoring(scf, 0.5)
        
        # 3. Hover at new position
        hover_at_position(scf, 3.0)
        
        # 4. Move forward again
        move_forward_with_position_monitoring(scf, 0.3)
        
        # 5. Hover again
        hover_at_position(scf, 2.0)
        
        # 6. Return to starting position
        return_to_start(scf)
        
        # 7. Final hover
        hover_at_position(scf, 2.0)
        
        print("Flight sequence completed successfully!")
        
    except Exception as e:
        print(f"Error during flight sequence: {e}")
        # Emergency landing
        print("Performing emergency landing...")
        with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
            mc.land()


## Execute Flight


In [ ]:
# Initialize the low-level drivers
cflib.crtp.init_drivers()

print(f"Connecting to Crazyflie at: {URI}")

with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
    # Set up deck detection callbacks
    scf.cf.param.add_update_callback(group='deck', name='bcLighthouse4', cb=param_deck_lighthouse)
    scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)

    # Proactively request initial values so callbacks fire
    for pname in ['deck.bcLighthouse4', 'deck.bcLighthouse']:
        try:
            scf.cf.param.request_param_update(pname)
        except Exception:
            pass
    for pname in ['deck.bcFlow2', 'deck.bcFlow']:
        try:
            scf.cf.param.request_param_update(pname)
        except Exception:
            pass
    """
    # Direct-read fallback in case callbacks did not fire
    try:
        lh_found = False
        for pname in ['deck.bcLighthouse4', 'deck.bcLighthouse']:
            try:
                val = scf.cf.param.get_value(pname)
                if val is not None and int(val) == 1:
                    lighthouse_deck_event.set()
                    lh_found = True
                    print(f"Lighthouse deck param detected: {pname}=1")
                    break
            except Exception:
                continue
        if not lh_found:
            print("Lighthouse param not confirmed yet; waiting for event...")
    except Exception:
        pass

    try:
        flow_found = False
        for pname in ['deck.bcFlow2', 'deck.bcFlow']:
            try:
                val = scf.cf.param.get_value(pname)
                if val is not None and int(val) == 1:
                    flow_deck_event.set()
                    flow_found = True
                    print(f"Flow deck param detected: {pname}=1")
                    break
            except Exception:
                continue
        if not flow_found:
            print("Flow param not confirmed yet; waiting for event...")
    except Exception:
        pass
    """
    # Set up position logging
    logconf = LogConfig(name='Position', period_in_ms=int(1000/UPDATE_RATE))
    logconf.add_variable('stateEstimate.x', 'float')
    logconf.add_variable('stateEstimate.y', 'float')
    logconf.add_variable('stateEstimate.z', 'float')
    scf.cf.log.add_config(logconf)
    logconf.data_received_cb.add_callback(log_position_callback)
    
    # Wait for both decks to be detected
    print("Waiting for decks to be detected...")
    if not lighthouse_deck_event.wait(timeout=10):
        print('ERROR: Lighthouse deck not detected!')
        # Print out all deck params to help debug
        try:
            print("Deck params:")
            for p in scf.cf.param.toc.toc:
                if p.startswith('deck.'):
                    try:
                        print(p, scf.cf.param.get_value(p))
                    except Exception:
                        print(p, 'N/A')
        except Exception:
            pass
        sys.exit(1)
    
    if not flow_deck_event.wait(timeout=10):
        print('ERROR: Flow deck not detected!')
        sys.exit(1)
    
    print("Both decks detected! Starting position logging...")
    
    # Start position logging
    logconf.start()
    
    # Wait a moment for initial position data
    time.sleep(2)
    
    # Arm the Crazyflie
    print("Arming Crazyflie...")
    scf.cf.platform.send_arming_request(True)
    time.sleep(1.0)
    
    # Execute main flight sequence
    main_flight_sequence(scf)
    
    # Stop logging
    logconf.stop()
    
    print("Flight completed!")


Connecting to Crazyflie at: radio://0/80/2M/E7E7E7E7E8
Flow deck is attached!
Lighthouse deck is attached!
Lighthouse deck param detected: deck.bcLighthouse4=1
Flow deck param detected: deck.bcFlow2=1
Waiting for decks to be detected...
Both decks detected! Starting position logging...
Lighthouse deck is attached!
Flow deck is attached!
Position: X=0.008, Y=0.136, Z=0.019
Position: X=0.008, Y=0.138, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0.008, Y=0.138, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0.008, Y=0.137, Z=0.018
Position: X=0.008, Y=0.137, Z=0.018
Position: X=0.008, Y=0.137, Z=0.018
Position: X=0.008, Y=0.136, Z=0.018
Position: X=0.007, Y=0.137, Z=0.018
Position: X=0.008, Y=0.138, Z=0.018
Position: X=0.008, Y=0.137, Z=0.018
Position: X=0.008, Y=0.138, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0.009, Y=0.138, Z=0.019
Position: X=0.008, Y=0.137, Z=0.019
Position: X=0

## Manual Control Functions

You can also use these functions individually for manual control:


In [ ]:
# Example: Manual takeoff and hover
# Uncomment the lines below to run individual functions

# with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
#     scf.cf.param.add_update_callback(group='deck', name='bcLighthouse4', cb=param_deck_lighthouse)
#     scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)
#     
#     logconf = LogConfig(name='Position', period_in_ms=20)
#     logconf.add_variable('stateEstimate.x', 'float')
#     logconf.add_variable('stateEstimate.y', 'float')
#     logconf.add_variable('stateEstimate.z', 'float')
#     scf.cf.log.add_config(logconf)
#     logconf.data_received_cb.add_callback(log_position_callback)
#     
#     if lighthouse_deck_event.wait(timeout=5) and flow_deck_event.wait(timeout=5):
#         logconf.start()
#         time.sleep(2)
#         scf.cf.platform.send_arming_request(True)
#         time.sleep(1.0)
#         
#         # Manual control examples:
#         # takeoff_and_hover(scf, 1.0)
#         # move_forward_with_position_monitoring(scf, 0.5)
#         # hover_at_position(scf, 5.0)
#         
#         logconf.stop()
#     else:
#         print('Required decks not detected!')


## Troubleshooting

**Common Issues:**

1. **Lighthouse deck not detected**: Ensure the Lighthouse deck is properly attached and the base stations are configured
2. **Flow deck not detected**: Check that the Flow deck is securely connected
3. **Position estimates not updating**: Verify that Lighthouse base stations are active and the drone is within range
4. **Connection issues**: Check your radio URI and ensure the Crazyflie is powered on

**Position Monitoring:**
- The notebook continuously logs XYZ position estimates from the Lighthouse system
- Position tolerance can be adjusted via the `HOVER_TOLERANCE` variable
- Update rate can be modified via the `UPDATE_RATE` variable

**Safety Features:**
- Emergency landing is triggered if errors occur during flight
- Position monitoring ensures movements are within expected ranges
- Deck detection prevents flight without required hardware
